In [1]:
%load_ext autoreload
%autoreload 2


import os
import re
import json
import glob
import random

import sys
sys.path.append('simulation')
from unity_simulator.comm_unity import UnityCommunication
from dataset_utils import execute_script_utils as utils
from evolving_graph import scripts

def setup():
    comm = UnityCommunication()
    return comm

/home/maithili/repos/tasksim/.venv/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


### Fetching all scripts and running them to filter the ones that run on sim

In [2]:
# DATASET = 'dataset/programs_processed'
# list_of_scene_scripts = glob.glob('{}/executable_programs/TrimmedTestScene{}_graph/*/*.txt'.format(DATASET,'1'))
def graph_file_from_script(script_file):
    return script_file.replace('executable_programs','init_and_final_graphs').replace('.txt','.json')

def process_file_inbuilt(script_file, graph_file):
    return utils.render_script_from_path(setup(),
                                script_file, graph_file,
                                {"processing_time_limit": 500, "image_width": 320, "image_height": 240, "image_synthesis": ['normal'], "gen_vid": True, "file_name_prefix": "test", "camera_mode": 'PERSON_TOP'}, scene_id=1)

In [4]:
# # # These were found to get the simulator stuck...
# list_of_scene_scripts.remove('dataset/programs_processed/executable_programs/TrimmedTestScene1_graph/results_text_rebuttal_specialparsed_programs_turk_third/split36_2.txt')
# list_of_scene_scripts.remove('dataset/programs_processed/executable_programs/TrimmedTestScene1_graph/results_intentions_march-13-18/file241_1.txt')
# list_of_scene_scripts.remove('dataset/programs_processed/executable_programs/TrimmedTestScene1_graph/results_intentions_march-13-18/file416_2.txt')
# list_of_scene_scripts.remove('dataset/programs_processed/executable_programs/TrimmedTestScene1_graph/results_intentions_march-13-18/file211_1.txt')
# list_of_scene_scripts.remove('dataset/programs_processed/executable_programs/TrimmedTestScene1_graph/results_intentions_march-13-18/file474_2.txt')
# list_of_scene_scripts.remove('dataset/programs_processed/executable_programs/TrimmedTestScene1_graph/results_intentions_march-13-18/file287_2.txt')
# list_of_scene_scripts.remove('dataset/programs_processed/executable_programs/TrimmedTestScene1_graph/results_intentions_march-13-18/file99_2.txt')
# list_of_scene_scripts.remove('dataset/programs_processed/executable_programs/TrimmedTestScene1_graph/results_intentions_march-13-18/file482_2.txt')
# list_of_scene_scripts.remove('dataset/programs_processed/executable_programs/TrimmedTestScene1_graph/results_intentions_march-13-18/file352_1.txt')
# list_of_scene_scripts.remove('dataset/programs_processed/executable_programs/TrimmedTestScene1_graph/results_intentions_march-13-18/file442_2.txt')
# list_of_scene_scripts.remove('dataset/programs_processed/executable_programs/TrimmedTestScene1_graph/results_text_rebuttal_specialparsed_programs_upwork_july/split23_9.txt')


In [5]:
# print('Total scripts to go through : ',len(list_of_scene_scripts))
# usable_scripts = []
# try:
#     for i,script_file in enumerate(list_of_scene_scripts):
#         res = process_file_inbuilt(script_file, graph_file_from_script(script_file))
#         success = res['success_expand']
#         if 'success_exec' in res.keys():
#             success = success and res['success_exec']
#         if success:
#             usable_scripts.append(script_file)
#         if i%50 == 0:
#             print(f'Processed {i} scripts')
# except Exception as e:
#     print(e)
# finally:
#     print(i,' files processed')
    # print('Last processed :',script_file)
    # print(len(usable_scripts))
    # with open(DATASET+'/usable_scripts.txt','a') as f:
    #     for s in usable_scripts:
    #         f.write(s+'\n')

### Categorize the usable script by activity and save in short_scripts

In [ ]:
# try:
#     assert(len(usable_scripts) > 0)
# except:
#     usable_scripts = []
#     with open(DATASET+'/usable_scripts.txt','r') as f:
#         for line in f:
#             usable_scripts.append(line[:-1])
# print(len(usable_scripts))

# activities = {}
# for script in usable_scripts:
#     with open(script,'r') as f:
#         activity = f.readline().strip()
#         if activity in activities:
#             activities[activity].append(script)
#         else:
#             activities[activity]=[script]
# for act in activities:
#     print (act,len(activities[act]))

### Fetching all scripts and running them to filter the ones that run on sim

In [ ]:
## Manual Work here!!!
# activity_categories = {}
# activity_categories['morning'] = ['Take shower', 'Turn on light', 'Open front door','Open door']
# activity_categories['night'] = ['Turn off light', 'Walk to room', 'Lock door']
# activity_categories['pastime'] = ['Read book', 'Read', 'Do work', 'Pet cat', 'Pick up phone', 'Go to toilet', 'Relax on sofa', 'Browse internet', 'Listen to music', 'Watch TV', 'Admire art']
# activity_categories['chore'] = ['Change sheets and pillow cases', 'Find dictionary','Put away shoes']
# activity_categories['food_pre'] = []
# activity_categories['food_post'] = ['Bring dirty plate to sink', ]
# activity_categories['unknown'] = ['Turn off TV', 'Bring me red cookbook', 'Close door', 'Keep cats out of room']

# activity_scripts = {key:[] for key in activity_categories.keys()}

# for act,scripts in activities.items():
#     for k,v in activity_categories.items():
#         if act in v:
#             activity_scripts[k] = activity_scripts[k] + scripts


In [ ]:
# from shutil import copyfile

# # for act, scripts in activity_scripts.items():
# #     os.makedirs('dataset/short_scripts/withoutconds/'+act)
# #     print(act, len(scripts))
# #     for script in scripts:
# #         script = script.replace('executable_programs/TrimmedTestScene1_graph','withoutconds')
# #         copyfile(script, 'dataset/short_scripts/withoutconds/'+act+'/'+os.path.basename(script))

# activity_scripts = {}
# base_dir = 'dataset/short_scripts/withoutconds/'
# for dir in glob.glob('{}/*'.format(base_dir)):
#     activity_scripts[os.path.basename(dir)] = glob.glob('{}/*.txt'.format(dir))

# for act,scripts in activity_scripts.items():
#     print (act, len(scripts))

### Generate concatenated scripts for full-day routine

In [ ]:
# routine_components = ['morning','night','pastime','chore','food_post']
# routine_sequence = ['morning','chore','pastime','food_post','pastime','chore','food_post','pastime','night']

# def write_script(script, file_name, script_name='', script_desc=''):
#     with open(file_name, 'w') as f:
#         f.write(script_name+'\n')
#         f.write(script_desc+'\n\n\n')
#         for s in script:
#             f.write(s+'\n')
 

# def sample_script():
#     script = []
#     for segment in routine_sequence:
#         segment_script_files = glob.glob('dataset/short_scripts/withoutconds/{}/*.txt'.format(segment))
#         file_name = random.choice(segment_script_files)
#         with open(file_name) as f:
#             for line in f:
#                 if '[' in line:
#                     script.append(line.strip())
#     return script


# !rm -r 'dataset/long_scripts'
# !mkdir -p 'dataset/long_scripts/withoutconds/batch1'
# for i in range(50):
#     write_script(sample_script(), 'dataset/long_scripts/withoutconds/batch1/'+str(i)+'.txt', 'Randomly generated routine', str(routine_sequence)[1:-1])

## Add preconditions for scripts + Create initial graphs and execute scripts

Scripts now run on their own init graph. How to generate an init graph for a sequence? Complete and fix the scripts manually then concatenate and try creating init graph and executable

In [3]:
import glob
path_input = 'dataset/my_scripts'
glob.glob('{}/withoutconds/*/*.txt'.format(path_input))

['dataset/my_scripts/withoutconds/breakfast_weekday_1/complete.txt',
 'dataset/my_scripts/withoutconds/laundry/complete.txt',
 'dataset/my_scripts/withoutconds/meal/complete.txt',
 'dataset/my_scripts/withoutconds/breakfast_weekday_2/complete.txt',
 'dataset/my_scripts/withoutconds/breakfast_weekend/complete.txt',
 'dataset/my_scripts/withoutconds/icecream/complete.txt',
 'dataset/my_scripts/withoutconds/sleep/complete.txt',
 'dataset/my_scripts/withoutconds/read/complete.txt',
 'dataset/my_scripts/withoutconds/meal_long/complete.txt']

In [4]:
### Test-A-Script

path_input = 'dataset/my_scripts'

from shutil import rmtree
if os.path.exists('{}/executable_programs'.format(path_input)):
    rmtree('{}/executable_programs'.format(path_input))
    rmtree('{}/init_and_final_graphs'.format(path_input))
    rmtree('{}/initstate'.format(path_input))
    rmtree('{}/state_list'.format(path_input))

# script_path = 'dataset/test_script/withoutconds/test/1.txt'
%run -i 'dataset_utils/add_preconds.py'
%run -i 'simulation/evolving_graph/check_programs.py'


Adding preconds for  dataset/my_scripts/withoutconds/breakfast_weekday_1/0.txt
Adding preconds for  dataset/my_scripts/withoutconds/breakfast_weekday_1/1.txt
Adding preconds for  dataset/my_scripts/withoutconds/breakfast_weekday_1/2.txt
Adding preconds for  dataset/my_scripts/withoutconds/breakfast_weekday_1/3.txt
Adding preconds for  dataset/my_scripts/withoutconds/breakfast_weekday_1/complete.txt
Adding preconds for  dataset/my_scripts/withoutconds/breakfast_weekend/0.txt
Adding preconds for  dataset/my_scripts/withoutconds/breakfast_weekend/1.txt
Adding preconds for  dataset/my_scripts/withoutconds/breakfast_weekend/2.txt
Adding preconds for  dataset/my_scripts/withoutconds/breakfast_weekend/3.txt
Adding preconds for  dataset/my_scripts/withoutconds/breakfast_weekend/complete.txt
Adding preconds for  dataset/my_scripts/withoutconds/icecream/0.txt
Adding preconds for  dataset/my_scripts/withoutconds/icecream/1.txt
Adding preconds for  dataset/my_scripts/withoutconds/icecream/2.txt
Ad

In [5]:
path_input = 'dataset/my_scripts/withoutconds/breakfast_weekday_1/0.txt'
# path_input = 'dataset/my_scripts/withoutconds/laundry/complete.txt'
# path_input = 'dataset/my_scripts/withoutconds/meal/complete.txt'
# path_input = 'dataset/my_scripts/withoutconds/breakfast_weekday_2/complete.txt'
# path_input = 'dataset/my_scripts/withoutconds/breakfast_weekend/complete.txt'
# path_input = 'dataset/my_scripts/withoutconds/icecream/complete.txt'
# path_input = 'dataset/my_scripts/withoutconds/sleep/complete.txt'
# path_input = 'dataset/my_scripts/withoutconds/read/complete.txt'
# path_input = 'dataset/my_scripts/withoutconds/meal_long/complete.txt'

translated_path = 'dataset/my_scripts/initial_common.json'

%run -i 'simulation/evolving_graph/check_programs.py'





100%|██████████| 1/1 [00:00<00:00, 4987.28it/s]

Total programs: 0, executable programs: 0 (unique: 0)
Programs that can not be parsed: 0 (unique: 0)
Executable program average length: 0.00, not executable program average length: 0.00


In [95]:
with open(translated_path,'r') as f:
    g1 = json.load(f)
# new_graph_path = 'dataset/my_scripts/temp.json'
new_graph_path = path_input.replace('withoutconds','init_and_final_graphs/initial_common').replace('.txt','.json')
print(new_graph_path)
with open (new_graph_path, 'r') as f:
    g2 = json.load(f)
g2 = g2['init_graph']
diff = {'nodes':{},'edges':{}}
def character_edge(e):
    return class_from_id(g1, e['from_id']) == 'character' or class_from_id(g1, e['to_id']) == 'character'
diff['edges']['removed'] = [e for e in g1['edges'] if e not in g2['edges'] and not character_edge(e)]
diff['edges']['added'] = [e for e in g2['edges'] if e not in g1['edges'] and not character_edge(e)]
diff['nodes']['removed'] = [n for n in g1['nodes'] if n['id'] not in [n2['id'] for n2 in g2['nodes']]]
diff['nodes']['added'] = [n for n in g2['nodes'] if n['id'] not in [n2['id'] for n2 in g1['nodes']]]
print(diff)

dataset/my_scripts/init_and_final_graphs/initial_common/read/complete.json
{'nodes': {'removed': [], 'added': []}, 'edges': {'removed': [], 'added': []}}


In [67]:
with open(translated_path,'w') as f:
    json.dump(g2, f)

In [3]:
# comm = setup()
# comm.reset(0)
# s, graph = comm.environment_graph()

# ignore_classes = ['floor','wall','ceiling']
# rooms = ['bathroom', 'bedroom', 'livingroom', 'kitchen']
# significant_classes = set()

def class_from_id(graph, id):
    lis = [n['class_name'] for n in graph['nodes'] if n['id']==id]
    if len(lis) > 0:
        return lis[0]
    else:
        return 'None'

def edge_string(graph, edge):
    from_class = class_from_id(graph, edge['from_id'])
    from_id = ' ('+str(edge['from_id'])+' )'
    to_class = class_from_id(graph, edge['to_id'])
    to_id = ' ('+str(edge['to_id'])+' )'
    return from_class+from_id+e['relation_type']+to_class+to_id



In [ ]:
import json
with open('dataset/my_scripts/initial_common.json','r') as f:
    my_graph = json.load(f)

print('Original my graph has ',len(my_graph['nodes']),' nodes and ',len(my_graph['edges']),' edges')

categories = set([n['category'] for n in my_graph['nodes']])
print(categories)


use_graph = {'nodes':[], 'edges':[]}
use_ids = []
class_ids = {}
for n in my_graph['nodes']:
    if n['category'] in ["Rooms", "Furniture", "Appliances", "placable_objects", "Characters"]:
        use_ids.append(n['id'])
        use_graph['nodes'].append(n)
    if n['class_name'] not in class_ids:
        class_ids[n['class_name']] = n['id']
for e in my_graph['edges']:
    if e['from_id'] in use_ids and e['to_id'] in use_ids:
        use_graph['edges'].append(e)

class_ids['kitchen'] = class_ids['dining_room']

print('My trimmed graph has ',len(use_graph['nodes']),' nodes and ',len(use_graph['edges']),' edges')
print (class_ids)

In [ ]:
room_node_ids = [n['id'] for n in my_graph['nodes'] if n['category'] == 'Rooms']
print(room_node_ids)
for e in my_graph['edges']:
    if e['to_id'] in room_node_ids:
        if e['relation_type'] != 'INSIDE':
            print(edge_string(my_graph,e))
    if e['from_id'] in room_node_ids:
            print(edge_string(my_graph,e))


In [ ]:
len(graph['nodes'])
for r in rooms:
    print()
    print(r)
    for e in graph['edges']:
        if class_from_id(graph, e['to_id']) == r and class_from_id(graph, e['from_id']) not in ignore_classes:
            print(class_from_id(graph, e['from_id']), end=' ')

In [ ]:
import json
significant_classes = set()
file_names = glob('dataset/good_scripts/state_list/TrimmedTestScene1_graph/*/complete.json')
wierd_classes = ['mat', 'drawing', 'toaster']

def get_difference(g1, g2):
    diff = {}
    def character_edge(e):
        return class_from_id(g1, e['from_id']) == 'character' or class_from_id(g1, e['to_id']) == 'character'
    diff['removed'] = [e for e in g1['edges'] if e not in g2['edges'] and not character_edge(e)]
    diff['added'] = [e for e in g2['edges'] if e not in g1['edges'] and not character_edge(e)]
    return diff

def print_graph_difference(g, diff):
    print('Removed...')
    for e in diff['removed']:
        print(edge_string(g, e))
    print('Added...')
    for e in diff['added']:
        print(edge_string(g, e))

def append_changed_classes(g, diff):
    for e in diff['removed']+diff['added']:
        if class_from_id(g, e['from_id']) in wierd_classes or class_from_id(g, e['to_id']) in wierd_classes:
            print(edge_string(g, e))
        significant_classes.add(class_from_id(g, e['from_id']))
        significant_classes.add(class_from_id(g, e['to_id']))

# for file_name in file_names:
#     with open(file_name) as f:
#         data = json.load(f)
#     data = data['graph_state_list']
#     print(file_name.split('/')[-2],' edges : ',len(data[0]['edges']),' nodes : ',len(data[0]['nodes']))
#     for i,(g1,g2) in enumerate(zip(data[0:-1],data[1:])):
#         append_changed_classes(g1, get_difference(g1, g2))
#     print_graph_difference(data[0], get_difference(data[0], data[-1]))


In [ ]:
from simulation.evolving_graph.execution import ScriptExecutor
from simulation.evolving_graph.utils import load_name_equivalence
from simulation.evolving_graph.check_programs import dump_one_data, modify_objects_unity2script

script_file_paths = glob.glob('dataset/my_scripts/withoutconds/*/complete.txt')

name_equivalence = load_name_equivalence()
complete_class_ids = class_ids.copy()
for c1,c2s in name_equivalence.items():
    id = complete_class_ids
    for c2 in c2s:
        if c1 in complete_class_ids:
            complete_class_ids[c2] = complete_class_ids[c1]
        elif c2 in complete_class_ids:
            complete_class_ids[c1] = complete_class_ids[c2]

for script_file in script_file_paths[:2]:
    # graph_dict = utils.load_graph_dict('dataset/my_scripts/initial_common.json')
    graph = EnvironmentGraph(my_graph)
    ## Need to make the script executable with ids before we execute
    name_equivalence = load_name_equivalence()
    executor = ScriptExecutor(graph, name_equivalence)
    script = read_script(script_file)
    executable, final_state, graph_state_list = executor.execute_script_file(script_file, w_graph_list=True, ids_by_class=complete_class_ids)
    if executable:
        print('Writing script outputs for : ',script_file)
        dump_one_data(script_file, script, graph_state_list, {}, 'something/custom_graph.json')

In [ ]:
name_equivalence['coffee']

In [ ]:
from simulation.evolving_graph.environment import EnvironmentGraph

print(len(graph['nodes']))
print(len(graph['edges']))
print(graph['nodes'][0])
classes = {}
for node in graph['nodes']:
    if node['class_name'] in classes:
        classes[node['class_name']] += 1
    else:
        classes[node['class_name']] = 1
print(sorted(list(classes.items())))

In [ ]:
from simulation.evolving_graph.environment import Relation
env = EnvironmentGraph(graph)
objects =  env.get_nodes_by_attr('class_name', 'shoes')
# print(f'{len(objects)} objects found!')
for obj in objects:
    print (type(obj))
    # stuff = [(r,vals) for (nid,r),vals in env._edge_map.items() if nid == l.id]
    # for s in stuff:
    #     print (s[0])
    #     print (s[1])
    #     print()
    # nodes = env.get_nodes_from(l, Relation.INSIDE)
    # for node in nodes:
    #     print (node.class_name)


In [ ]:
# import simulation.evolving_graph.utils as utils
# from simulation.evolving_graph.scripts import read_script_from_list_string
# from simulation.evolving_graph.preparation import AddMissingScriptObjects, AddRandomObjects, ChangeObjectStates
# from simulation.evolving_graph.execution import ScriptExecutor, Action

# script_list = ['[Walk] <bathroom> (1)', '[Walk] <shower> (2)', '[Find] <shower> (2)']

# graph = utils.load_graph('example_graphs/TestScene1_graph.json')
# script = read_script_from_list_string(script_list)
# for l in script:
#     print (str(l))

# name_equivalence = utils.load_name_equivalence()
# object_placing = utils.load_object_placing()
# properties_data = utils.load_properties_data()
# executor = ScriptExecutor(graph, name_equivalence)

# # Execute script; fails due to a missing object
# # state_enum = executor.find_solutions(script)
# # state = next(state_enum, None)
# # print('Script is {0}executable'.format('not ' if state is None else ''))

# # Add missing objects (random)
# prepare_1 = AddMissingScriptObjects(name_equivalence, properties_data, object_placing)
# print(prepare_1)

# # Add 10 random objects
# prepare_2 = AddRandomObjects(properties_data, object_placing, choices=10)

# # Change states of "can_open" and "has_switch" objects to
# # open/closed, on/off)
# prepare_3 = ChangeObjectStates(properties_data)

# state_enum = executor.find_solutions(script, [prepare_1, prepare_2, prepare_3])
# state = next(state_enum, None)
# print('Script is {0}executable'.format('not ' if state is None else ''))

In [ ]:
!mkdir dataset/long_scripts_/videos
script_file = 'dataset/short_scripts/executable_programs/TrimmedTestScene1_graph/chore/1.txt'
process_file_inbuilt(script_file, graph_file_from_script(script_file))
for script_file in glob.glob('dataset/long_scripts_/executable_programs/TrimmedTestScene1_graph/batch1/*.txt'):
    print(process_file_inbuilt(script_file, graph_file_from_script(script_file)))
    file_num = os.path.splitext(os.path.basename(script_file))[0]
    copyfile('Output/test/Action_normal.mp4', 'dataset/long_scripts_/videos/'+file_num+'.mp4')

In [ ]:
import simulation.evolving_graph.utils as utils
from simulation.evolving_graph.scripts import read_script
from simulation.evolving_graph.execution import ScriptExecutor

for script_file in glob.glob('dataset/long_scripts/executable_programs/TrimmedTestScene1_graph/batch1/*.txt'):
    graph = utils.load_graph(graph_file_from_script(script_file))
    script = read_script('example_scripts/example_script_3.txt')
    name_equivalence = utils.load_name_equivalence()
    executor = ScriptExecutor(graph, name_equivalence)
    success, state, graph_state_list = executor.execute(script)
    if not success:
        print('Script is not executable, since {}'.format(executor.info.get_error_string()))
    else:
        print('Script is executable')
